# Movie Advisor
*Chat with an AI movie advisor and get recommendations based on your preferences.*

[GitHub Repo](https://github.com/alexdjulin/movie-advisor)

## Brainstorming

Goal of this project is to get my hands on langchain and build an AI movie advisor. I want to be able to do the following:
- Ask for information about a movie ('I heard The Godfather is good, give me the plot')
- Ask for recommendations based on given criterias ('I like adventure movies, especially on the time-travel subject. Could you recommend a few?')
- Handle a list of movies I already watched and if I liked them or not, use it to fine-tune recommendations
- Handle a must-see and not-interested lists of movies, add to / remove movies from it
- Use RAG techniques to access information outside of the LLM knowledge: Search for a recent or less-known movie on the Internet or on a movie database

Interactions with the movie advisor should be via keyboard input first, then using speech.

The AI part should rely on a LLM. I will use OpenAI Gpt-4o at first, then try implementing open-source models.

Source info should come from the LLM first. I will try to pack the imdb dataset in a database for the model to consult, but I guess that it's been trained on it already. Still, it would be interesting to see if the model can add entries to it, for recent or less-known movies it has not heard about yet.

Lists should be stored in a database. I will use [Xata](https://app.xata.io/) for this.


## Install modules

In [114]:
! pip install -qU python-dotenv langchain langchain-community langchainhub openai langchain-openai ipykernel kaggle pandas xata requests tiktoken langchain_huggingface sentence-transformers

## Imports

In [14]:
# misc
import os
import zipfile
import pandas as pd
# langchain
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import tool
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.prompts import ChatPromptTemplate
from langchain_community.vectorstores.xata import XataVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.schema import Document
from langchain_core.pydantic_v1 import BaseModel, Field
# openai
embeddings = OpenAIEmbeddings()
# xata client and config
from xata.client import XataClient
xata = XataClient()
# load environment variables
import dotenv
dotenv.load_dotenv()

sep = 100 * "-"

## Dataset review

Here are some databases on Kaggle that our LLM could access to or extend:
- [IMDB Movies Dataset](https://www.kaggle.com/datasets/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows) - 1000 entries from the IMDB database
- [TMDB 5000 Movie Dataset](https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata) - 5000 entries from the TMDB database ()
- [Netflix Movies and TV Shows](https://www.kaggle.com/datasets/shivamb/netflix-shows) - Netflix contents, probably a bit too restrictive

Let's have a look at the IMDB one. 1000 entries should be enough to start.

## Review the IMDB Movie Dataset

### Download

In [5]:
!kaggle datasets download -d tmdb/tmdb-movie-metadata

dataset_path = 'dataset'
zip_file = 'tmdb-movie-metadata.zip'

# unzip dataset
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(dataset_path)

os.remove(zip_file)
    

Dataset URL: https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata
License(s): other




  0%|          | 0.00/8.89M [00:00<?, ?B/s]
 11%|█▏        | 1.00M/8.89M [00:00<00:04, 1.79MB/s]
 23%|██▎       | 2.00M/8.89M [00:00<00:02, 3.50MB/s]
 45%|████▌     | 4.00M/8.89M [00:00<00:00, 6.24MB/s]
 68%|██████▊   | 6.00M/8.89M [00:01<00:00, 7.95MB/s]
 90%|█████████ | 8.00M/8.89M [00:01<00:00, 9.10MB/s]
100%|██████████| 8.89M/8.89M [00:01<00:00, 7.12MB/s]


### Extract data we need
The database has extensive informaton. We will only extract the following columns:
- title (string)
- genres (list)
- keywords (list)
- overview (string)
- vote average (float)
- production_countries
- release_date (string or datetime)
- runtime in minutes (ing)

In [6]:
# load dataset and print columns
tmdb_df = pd.read_csv(f'{dataset_path}/tmdb_5000_movies.csv')
print(sorted(tmdb_df.columns.tolist()))

['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language', 'original_title', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'vote_average', 'vote_count']


In [9]:
# filter columns to keep and make a copy of the df
columns = ["title", "genres", "keywords", "overview", "vote_average", "production_countries", "release_date", "runtime"]
df = tmdb_df.copy().dropna()
df = df[columns]

# format columns with multiple values as lists
for column in ["genres", "keywords", "production_countries"]:
    df[column] = df[column].apply(lambda x: [i['name'] for i in eval(x)])

# # format floats to strings
# for column in ["vote_average", "runtime"]:
#     df[column] = df[column].astype(str)

# make sure all columns are strings
for column in columns:
    col = df[column][0]
    print(type(col), col)
    print()

df.head()

<class 'str'> Avatar

<class 'list'> ['Action', 'Adventure', 'Fantasy', 'Science Fiction']

<class 'list'> ['culture clash', 'future', 'space war', 'space colony', 'society', 'space travel', 'futuristic', 'romance', 'space', 'alien', 'tribe', 'alien planet', 'cgi', 'marine', 'soldier', 'battle', 'love affair', 'anti war', 'power relations', 'mind and soul', '3d']

<class 'str'> In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.

<class 'numpy.float64'> 7.2

<class 'list'> ['United States of America', 'United Kingdom']

<class 'str'> 2009-12-10

<class 'numpy.float64'> 162.0



,title,genres,keywords,overview,vote_average,production_countries,release_date,runtime
0,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",7.2,"[United States of America, United Kingdom]",2009-12-10,162.0
1,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",6.9,[United States of America],2007-05-19,169.0
2,Spectre,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,6.3,"[United Kingdom, United States of America]",2015-10-26,148.0
3,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...,7.6,[United States of America],2012-07-16,165.0
4,John Carter,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca...",6.1,[United States of America],2012-03-07,132.0


## Create Xata TMDB movies table

In [10]:
def create_table(table_name: str, table_schema: dict) -> None:
    """ create table if it doesn't exist """
    try:
        assert xata.table().create(table_name).is_success()
    except AssertionError as e:
        print(f"Error creating table '{table_name}': {e}")
    try:
        resp = xata.table().set_schema(table_name, table_schema)
        assert resp.is_success(), resp
    except AssertionError as e:
        print(f"Error setting schema for table '{table_name}': {e}")

In [11]:
table_name = "tmdb_movies"

table_schema = {
    "columns": [
        {"name": "title", "type": "string"},
        {"name": "genres", "type": "multiple"},
        {"name": "keywords", "type": "multiple"},
        {"name": "overview", "type": "string"},
        {"name": "vote_average", "type": "float"},
        {"name": "production_countries", "type": "multiple"},
        {"name": "release_date", "type": "string"},
        {"name": "runtime", "type": "float"},
    ]
}

create_table(table_name, table_schema)

# insert one entry as a test
record = df.iloc[0].to_dict()
print(record)

xata.records().insert(table_name, record)

{'title': 'Avatar', 'genres': ['Action', 'Adventure', 'Fantasy', 'Science Fiction'], 'keywords': ['culture clash', 'future', 'space war', 'space colony', 'society', 'space travel', 'futuristic', 'romance', 'space', 'alien', 'tribe', 'alien planet', 'cgi', 'marine', 'soldier', 'battle', 'love affair', 'anti war', 'power relations', 'mind and soul', '3d'], 'overview': 'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.', 'vote_average': 7.2, 'production_countries': ['United States of America', 'United Kingdom'], 'release_date': '2009-12-10', 'runtime': 162.0}


{'id': 'rec_cqd5rbui9lerrvc51hd0',
 'xata': {'createdAt': '2024-07-19T12:35:59.644741Z',
  'updatedAt': '2024-07-19T12:35:59.644741Z',
  'version': 0}}

In [12]:
# check if table exists
table_name = "tmdb_movies"
assert xata.data().query(table_name).is_success()

### Fill up table

In [ ]:
# if successful, fill up database
table_name = "tmdb_movies"

# METHOD 1: isert records one by one
# records = df.to_dict(orient='records')
# for record in records:
    # xata.records().insert(table_name, record)

# METHOD 2: using bulk insert / faster but 1000 records limit at a time
xata.records().bulk_insert(table_name, {"records": df.to_dict(orient='records')[:999]})
xata.records().bulk_insert(table_name, {"records": df.to_dict(orient='records')[1000:]})

The dataset is now on xata.

## Create movie history table
Let's now create a second table to store my movie history:
- Title (str): movie Title
- Status (str): watched, must_see, not_interested
- Comment (str): some personal comment about it
- Content (str): a short text summing up (title, status, comment) that will be used as embedding
- Embedding (vector): an OpenAI embedding vector of the content string


In [18]:
table_name = "movie_history"
table_schema = {
    "columns": [
        {"name": "title", "type": "text"},
        {"name": "status", "type": "text"},
        {"name": "comment", "type": "text"},
        {"name": "content", "type": "text"},
        {"name": "embedding", "type": "vector", "vector": {"dimension": 1536}}
    ]
}

# create table
# create_table(table_name, table_schema)

# create table vectorstore
table_name = "movie_history"
vector_store = XataVectorStore(
    embedding=embeddings,
    api_key=os.getenv("XATA_API_KEY"),
    db_url=os.getenv("XATA_DATABASE_URL"),
    table_name=table_name
)

vector_store


### Query table records

In [45]:
# query the content of movie_history
table_name = "movie_history"

def get_table_records(table_name: str) -> list:
    records = xata.data().query(table_name)["records"]
    return records

records = get_table_records(table_name)

if not records:
    print("No records found")
else:    
    for rec in records:
        print("id: ", rec["id"])
        print("Title: ", rec["title"])
        print("Status: ", rec["status"])
        print("Comment: ", rec["comment"])
        print("Content: ", rec["content"])
        print(sep)



id:  rec_cqd614qm85ansdaq79c0
Title:  Inception
Status:  watched
Comment:  This is one of my favourite movie, especially how it handles time paradox!
Content:  Inception (watched) one of my favourite movie, liked the time paradox
----------------------------------------------------------------------------------------------------
id:  rec_cqd614qm85ansdaq79cg
Title:  Jurassik Park
Status:  watched
Comment:  A true masterpiece. This is probably the best movie of all times, I could watch it forever and never get bored
Content:  Jurassik Park (watched) best movie of all times, masterpiece, never get bored
----------------------------------------------------------------------------------------------------
id:  rec_cqd6152m85ansdaq79d0
Title:  Once Upon a Time in America
Status:  must_see
Comment:  I have been willing to watch it for a long time, but it's a long movie and I never found the time to watch it
Content:  Once Upon a Time in America (must_see) Very long, never found the time to wa

### Fill up table with some test records

In [44]:
def add_update_movie(movie_record: dict) -> None:

    table_records = get_table_records(table_name)

    # delete record if it exists
    for rec in table_records:
        if rec["title"] == movie_record["title"]:
            xata.records().delete(table_name, rec["id"])
            break

    # add updated record
    doc = Document(page_content=movie_record["content"], metadata={k: v for k, v in movie_record.items() if k != "content"})
    vector_store.add_documents([doc])

In [25]:
rec_1 = {
    "title": "Inception",
    "status": "watched",
    "comment": "This is one of my favourite movie, especially how it handles time paradox!",
    "content": "Inception (watched) one of my favourite movie, liked the time paradox"
}

rec_2 = {
    "title": "Jurassic Park",
    "status": "watched",
    "comment": "A true masterpiece. This is probably the best movie of all times, I could watch it forever and never get bored",
    "content": "Jurassik Park (watched) best movie of all times, masterpiece, never get bored"
}

rec_3 = {
    "title": "West Side Story",
    "status": "must_see",
    "comment": "I heard it's a very good movie and the music is amazing. I can't wait to watch it",
    "content": "West Side Story (must_see) music amazing, can't wait to watch it"
}

rec_4 = {
    "title": "Once Upon a Time in America",
    "status": "must_see",
    "comment": "I have been willing to watch it for a long time, but it's a long movie and I never found the time to watch it",
    "content": "Once Upon a Time in America (must_see) Very long, never found the time to watch it"
}

rec_5 = {
    "title": "The Exorcist",
    "status": "not_interested",
    "comment": "I don't like horror movies and I heard this one is pretty rough, I prefer not to watch it",
    "content": "The Exorcist (not on watch_list) I dont like horror movies"
}

for rec in [rec_1, rec_2, rec_3, rec_4, rec_5]:
    add_update_movie(rec)


### Add a new record or update an existing one in the table

In [38]:
table_name = "movie_history"

rec_6 = {
    "title": "eXistenZ",
    "status": "watched",
    "comment": "I didnt like it, I found it very confusing and the story was sometimes disturbing",
    "content": "eXistenZ (watched) Didnt like it, confusing, disturbing"
}

rec_7 = {
    "title": "West Side Story",
    "status": "watched",
    "comment": "I finally watched it last night, it was amazing, thrilling music",
    "content": "West Side Story (watched) amazing, thrilling music"
}

for rec in [rec_6, rec_7]:
    add_update_movie(rec)


### Delete a record from the table

In [47]:
table_name = "movie_history"

def delete_movie_from_table(title: str) -> None:
    records = get_table_records(table_name)
    for rec in records:
        if rec["title"] == title:
            xata.records().delete(table_name, rec["id"])
            print(f"Record '{title}' deleted successfully")
            return
    print(f"Record '{title}' not found")

delete_movie_from_table("Inception")

Record 'Inception' deleted successfully


In [48]:
def get_watch_lists(table_name: str) -> dict:
    """
    Return the list of watched, not watched and blacklists movies in a dict
    """
    
    records = get_table_records(table_name)

    watched = [rec['title'] for rec in records if rec["status"] == "watched"]
    must_see = [rec['title'] for rec in records if rec["status"] == "must_see"]
    not_interested = [rec['title'] for rec in records if rec["status"] == "not_interested"]

    return {"watched": watched, "must_see": must_see, "not_interested": not_interested}

table_name = "movie_history"
watch_dict = get_watch_lists(table_name)
print(watch_dict)

{'watched': ['Jurassik Park', 'West Side Story'], 'must_see': ['Once Upon a Time in America'], 'not_interested': ['The Exorcist']}


### Similarity Search test
Let's see if we can extract information from our history

In [50]:
query = "What movies am I most affraid of?"
found_docs = vector_store.similarity_search(query, k=1)
print(found_docs[0].page_content)

query = "What is my favourite movie of all times?"
found_docs = vector_store.similarity_search(query, k=1)
print(found_docs[0].page_content)

The Exorcist (not on watch_list) I dont like horror movies
Jurassik Park (watched) best movie of all times, masterpiece, never get bored


## Build our LLM structure

### Prompt

In [9]:
template = """You are a movie advisor offering me recommendations based on my preferences and watch history from these 3 lists:
- Watched: Movies I have already seen: {watched}.
- Watchlist: Movies I haven't seen yet but I want to: {watchlist}.
- Blacklist: Movies I haven't seen yet but I don't want to: {blacklist}.
Only recommend one movie at a time.
My preferences related to the question to orient your answer: {preferences}.
Question: {question}.
"""

prompt = ChatPromptTemplate.from_template(template)
print(prompt)


input_variables=['blacklist', 'preferences', 'question', 'watched', 'watchlist'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['blacklist', 'preferences', 'question', 'watched', 'watchlist'], template="You are a movie advisor offering me recommendations based on my preferences and watch history from these 3 lists:\n- Watched: Movies I have already seen: {watched}.\n- Watchlist: Movies I haven't seen yet but I want to: {watchlist}.\n- Blacklist: Movies I haven't seen yet but I don't want to: {blacklist}.\nOnly recommend one movie at a time.\nMy preferences related to the question to orient your answer: {preferences}.\nQuestion: {question}.\n"))]


### Similarity Retriever

In [10]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 1})
results = retriever.invoke("I'm affraid of horror movies")
print(results[0].page_content)

The Exorcist (not on watch_list) I dont like horror movies, prefer not to watch it


### LLM Model

In [22]:
llm_gpt4 = ChatOpenAI(model='gpt-4o-mini')

### String output parser
Create a string output parser for general chatting with the advisor about movies

In [23]:
str_output_parser = StrOutputParser()

In [14]:
# test string parser
test_prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": "String"},
)

str_chain = test_prompt | llm_gpt4 | str_output_parser
response = str_chain.invoke({"query": "What movie should I watch tonight?"})
print(response)

It depends on what genre you're in the mood for! Here are a few suggestions:

- **Action/Adventure**: *Mad Max: Fury Road* - A visually stunning, high-octane ride through a post-apocalyptic world.
- **Comedy**: *Superbad* - A coming-of-age teen comedy that’s both hilarious and relatable.
- **Drama**: *The Shawshank Redemption* - A powerful story about hope and friendship set in a prison.
- **Horror**: *Get Out* - A thought-provoking thriller that combines social commentary with suspense.
- **Romance**: *La La Land* - A modern musical that explores love and ambition in Los Angeles.

Let me know if you want more specific recommendations or if you have a particular genre in mind!


### JSON Output Parser
Create a json output parser to update the database
[Documentation](https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/types/json/)

In [15]:
model = ChatOpenAI(temperature=0)

# define desired data structure
class Recommendation(BaseModel):
    title: str = Field(description="The movie title you recommend")
    comment: str = Field(description="My personal opinion about the movie")
    watched: bool = Field(description="True to add it to the list of watched movies, else False")
    watchlist: bool = Field(description="True to add it to the list of movies I would like to watch, else False")
    blacklist: bool = Field(description="True to add it to the list of movies I don't want to watch, else False")
    embedding: bool = Field(description="A short text that describes my answer as follow: {movie_title} ({watched/watchlist/blacklist}) {personal_opinion}. It will be stored as an embedding vector in the database")

json_output_parser = JsonOutputParser(pydantic_object=Recommendation)
print(json_output_parser)

pydantic_object=<class '__main__.Recommendation'>


In [16]:
# test json parser
test_prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": json_output_parser.get_format_instructions()},
)

json_chain = test_prompt | model | json_output_parser
json_chain.invoke({"query": "What movie should I watch tonight?"})

{'title': 'Inception',
 'comment': "One of the best mind-bending movies I've ever seen",
 'watched': False,
 'watchlist': True,
 'blacklist': False,
 'embedding': "Inception (watchlist) One of the best mind-bending movies I've ever seen"}

### Build chain

In [ ]:
# define input
table_name = "movie_history"
history = get_movie_history(table_name)

watched = (lambda x: history["watched"])
watchlist = (lambda x: history["watchlist"])
blacklist = (lambda x: history["blacklist"])

preferences = (lambda x: x["question"]) | retriever

question = (lambda x: x["question"])

input_var = {"watched": watched, "watchlist": watchlist, "blacklist": blacklist, "preferences": preferences, "question": question}

# define chain
chain = (input_var | prompt | llm_gpt4 | str_output_parser)

# invoke answer
answer = chain.invoke({"question": "Could you recommand a good horror movie?"})
print(answer)

### Build Tools
Let's build tool methods that our model can use to add or remove a movie from a list.

In [ ]:
def remove_title_from_all_lists(title):
    """Make sure a title is in none of the lists"""
    global watch_dict
    for titles in watch_dict.values():
        if title in titles:
            titles.remove(title)

@tool
def add_title_to_list_of_movies_I_have_already_watched(title: str) -> None:
    """Add a movie title to the list of movies I have already watched in the past"""
    global watch_dict
    if title not in watch_dict["watched"]:
        remove_title_from_all_lists(title)
        watch_dict["watched"].append(title)

@tool
def add_title_to_list_of_movies_I_want_to_watch_later(title: str) -> None:
    """Add a movie title to the list of movies I want to watch later"""
    global watch_dict
    if title not in watch_dict["must_see"]:
        remove_title_from_all_lists(title)
        watch_dict["must_see"].append(title)

@tool
def add_title_to_list_of_movies_I_never_want_to_watch(title: str) -> None:
    """Add a movie title to the list of movies I never want to watch"""
    global watch_dict
    if title not in watch_dict["not_interested"]:
        remove_title_from_all_lists(title)
        watch_dict["not_interested"].append(title)

@tool
def remove_title_from_list_of_movies_I_have_already_watched(title: str) -> None:
    """Remove a movie title from the list of movies I have already watched"""
    global watch_dict
    if title in watch_dict["watched"]:
        watch_dict["watched"].remove(title)

@tool
def remove_title_from_list_of_movies_I_want_to_watch_later(title: str) -> None:
    """Remove a movie title from the list of movies I want to see"""
    global watch_dict
    if title in watch_dict["must_see"]:
        watch_dict["must_see"].remove(title)

@tool
def add_title_from_list_of_movies_I_never_want_to_watch(title: str) -> None:
    """Remove a movie title from the list of movies I'm not interested in"""
    global watch_dict
    if title in watch_dict["not_interested"]:
        watch_dict["not_interested"].remove(title)


# List of tools
tools = [
    add_title_to_list_of_movies_I_have_already_watched,
    add_title_to_list_of_movies_I_want_to_watch_later,
    add_title_to_list_of_movies_I_never_want_to_watch,
    remove_title_from_list_of_movies_I_have_already_watched,
    remove_title_from_list_of_movies_I_want_to_watch_later,
    add_title_from_list_of_movies_I_never_want_to_watch
]

# link our model to the tools
gpt4_with_tools = llm_gpt4.bind_tools(tools) 


### Create Agent

Let's now create an agent that will use the tools to update our lists.

In [126]:
# import premade prompt
prompt = hub.pull("hwchase17/openai-tools-agent")
print(prompt)

agent = create_tool_calling_agent(llm_gpt4, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

watch_dict = {"watched": [], "must_see": [], "not_interested": []}
print(watch_dict)

result = agent_executor.invoke(
    {
        "input": "Could you please add the movie 'The Matrix' to my watched list?"
    }
)

print(result)
print(watch_dict)

input_variables=['agent_scratchpad', 'input'] optional_variables=['chat_history'] input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} partial_variables={'chat_history': []} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-tools-agent', 'lc_hub_commit_hash': 'c18672812789a3b9697656dd539edf0120285dcae36396d0b548ae42a4ed66f5'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], templa

In [ ]:
# test it in a while loop
watch_dict = {"watched": [], "must_see": [], "not_interested": []}

while True:

    print("WATCH STATUS:", watch_dict)
    
    input_text = input("Alex:")
    if not input_text:
        break

    result = agent_executor.invoke({"input": input_text})
    print("Agent:", result)

## Query the TMDB database
Let's use langchain tool to query the TMDB database for a movie that our LLM would potentially not know about.  

We can see below that gpt-4o has not heard about the latest marvel movie 'Deadpool & Wolverine', while it is already in the TMDB database [here](https://www.themoviedb.org/movie/533535-deadpool-wolverine).

In [30]:
# Let's find a movie that gpt4o does not know about
test_prompt = PromptTemplate(
    template="You are a movie advisor, you answer questions about movies.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": "String"},
)

str_chain = test_prompt | llm_gpt4 | str_output_parser
response = str_chain.invoke({"query": "Give me the plot of the movie 'Deadpool & Wolverine'?"})
print(response)

As of my last update in October 2023, "Deadpool & Wolverine" is not an officially released movie, but it is often speculated about due to the popularity of both characters in the Marvel universe. However, there is an anticipated film titled "Deadpool 3," which is expected to feature both Deadpool (played by Ryan Reynolds) and Wolverine (played by Hugh Jackman).

In the context of the expected plot, it is likely to involve Deadpool's irreverent humor and fourth-wall-breaking antics as he teams up (or clashes) with Wolverine, known for his more serious demeanor. The storyline may include elements of action, comedy, and possibly a multiverse theme, considering Deadpool's unpredictable nature and Wolverine's history with various X-Men storylines.

If you're looking for the latest updates or specific plot details for a movie that might have been released or announced after October 2023, I recommend checking the latest entertainment news or official movie databases.


Now let's create an agent with access to the tmdb-api tool, so gpt4o can do api calls to the TMDB database when needed.

You will need a TMDB account with an API-key for this.  

[TMDP API Documentation](https://developer.themoviedb.org/docs/getting-started)

### Let's check if api key is working

In [2]:
import requests

def search_movie(api_key, query):
    # Base URL for the search endpoint
    url = "https://api.themoviedb.org/3/search/movie"
    
    # Parameters for the API request
    params = {
        'api_key': api_key,
        'query': query
    }
    
    # Making the GET request to the API
    response = requests.get(url, params=params)

    # Checking if the request was successful
    if response.status_code == 200:
        # Parsing the JSON response
        data = response.json()
        return data['results']
    else:
        # If the request failed, print the status code
        print(f"Error: {response.status_code}")
        return None

# Example usage
api_key = os.getenv("TMDB_BEARER_TOKEN")
query = "Inception"

results = search_movie(api_key, query)

if results:
    for movie in results:
        print(f"Title: {movie['title']}, Release Date: {movie['release_date']}, Overview: {movie['overview']}")
else:
    print("No results found.")


Title: Inception, Release Date: 2010-07-15, Overview: Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inception", the implantation of another person's idea into a target's subconscious.
Title: The Crack: Inception, Release Date: 2019-10-04, Overview: Madrid, Spain, 1975; shortly after the end of the Franco dictatorship. Six months after the mysterious death of his lover, a prestigious tailor, a married woman visits the office of the young Germán Areta, a former police officer turned private detective, to request his professional services.
Title: Inception: The Cobol Job, Release Date: 2010-12-07, Overview: This "Inception" prequel unfolds courtesy of a beautiful Motion Comic, and explains how Cobb, Arthur and Nash were enlisted by Cobol Engineering.
Title: The Inception of Dramatic Representation, Release Date: , Overview: On certain Adavu

### Using Langchain tmdb-api tool
So far I could not get this tool to work. It seems that it's not adding my api-key to the request url and I get an error.

In [ ]:
import os
from langchain.agents import AgentExecutor, create_tool_calling_agent, load_tools
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# load api-keys from environment variables or specify them below
openai_api_key = os.getenv("OPENAI_API_KEY")
tmdb_bearer_token = os.getenv("TMDB_BEARER_TOKEN")

# define the language model
llm = ChatOpenAI(model='gpt-4o', api_key=openai_api_key)

# load the tmdb-api tool
tools = load_tools(["tmdb-api"], llm=llm, tmdb_bearer_token=tmdb_bearer_token)

# define the prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie advisor, you answer questions about movies."),
        ("system", "If you don't know an answer, invoke the TMDB-API with a question in natural language."),
        ("human", "{query}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# create the agent
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# test the agent
result = agent_executor.invoke(
    {
        "query": "Give me the plot of movie 'Inception'?",
    }
)

# print the result
print("Agent: ", result["output"])

In [8]:
from langchain_community.utilities.requests import TextRequestsWrapper

# Create a TextRequestsWrapper instance with custom headers
headers = {"Authorization": f"Bearer {os.getenv('TMDB_BEARER_TOKEN')}"}
requests_wrapper = TextRequestsWrapper(headers=headers)

# Make a GET request and inspect the response
response = requests_wrapper.get(f"https://api.themoviedb.org/3/search/movie?query=Inception&language=en-US&api_key={os.getenv('TMDB_BEARER_TOKEN')}")
print(response)

{"page":1,"results":[{"adult":false,"backdrop_path":"/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg","genre_ids":[28,878,12],"id":27205,"original_language":"en","original_title":"Inception","overview":"Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: \"inception\", the implantation of another person's idea into a target's subconscious.","popularity":134.447,"poster_path":"/edv5CZvWj09upOsy2Y6IwDhK8bt.jpg","release_date":"2010-07-15","title":"Inception","video":false,"vote_average":8.369,"vote_count":36033},{"adult":false,"backdrop_path":"/JeGkRdNsOuMrgwBdtB0hp763MU.jpg","genre_ids":[18,53],"id":613092,"original_language":"es","original_title":"El crack cero","overview":"Madrid, Spain, 1975; shortly after the end of the Franco dictatorship. Six months after the mysterious death of his lover, a prestigious tailor, a married woman visits the office of the yo

## Audio Parsers
Let's see if we can add the STT and TTS steps to the chain

In [23]:
!pip install --upgrade --quiet langchain-google-community[speech]
!pip install -upgrade --quiet langchain-google-community


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -u


In [ ]:
from langchain_community.document_loaders.parsers.audio import OpenAIWhisperParserLocal

project_id = "movie-advisor"
file_path = "prototypes/audio.wav"

loader = GoogleSpeechToTextLoader(project_id=project_id, file_path=file_path)

docs = loader.load()

print(docs[0].page_content)
print